In [ ]:
import pandas as pd
import numpy as np

def simple_exponential_smoothing(data, alpha):
    smoothed = [data[0]]
    for t in range(1, len(data)):
        if data[t] == '?':  # Replace '?' with the previous value
            data[t] = data[t - 1]
        smoothed_value = alpha * float(data[t]) + (1 - alpha) * smoothed[t - 1]
        smoothed.append(smoothed_value)
    return smoothed

file_path = input("Enter the path to the .csv file: ")
data_column = input("Enter the column name for time series data: ")
data = pd.read_csv(file_path)
forecast_periods = int(input("Enter the number of periods to forecast: "))

if data_column not in data.columns:
    print(f"Column '{data_column}' not found in the file.")
    exit()

time_series_data = data[data_column].replace('?', np.nan).to_list()
time_series_data = [str(x) if x == '?' else pd.to_numeric(x, errors='coerce') for x in time_series_data]

numeric_data = [x for x in time_series_data if x != '?']
if len(numeric_data) < 1:
    print("The selected column does not contain any valid numeric data.")
    exit()

alpha = float(input("Enter alpha (0 < alpha <= 1): "))
if not (0 < alpha <= 1):
    print("Invalid alpha value. Must be between 0 and 1.")
    exit()

time_series_data = [numeric_data[0] if i == 0 else time_series_data[i] for i in range(len(time_series_data))]
for i in range(1, len(time_series_data)):
    if time_series_data[i] == '?':
        time_series_data[i] = time_series_data[i - 1]  # Replace '?' with the previous value
ses_result = simple_exponential_smoothing(time_series_data, alpha)


forecast_values = []
last_smoothed_value = ses_result[-1]  # Last smoothed value

for _ in range(forecast_periods):
    forecast_value = alpha * last_smoothed_value + (1 - alpha) * last_smoothed_value
    forecast_values.append(forecast_value)
    last_smoothed_value = forecast_value
data[data_column] = [str(x) if x == '?' else x for x in time_series_data]

results = pd.DataFrame({
    'Original': time_series_data + ['Forecast'] * forecast_periods,  # Add 'Forecast' label
    'SES': ses_result + forecast_values  # Append forecast values
})

print("\nSmoothed Results:")
print(results)

output_file = input("Enter the output file name (with .csv extension): ").strip()
try:
    data.to_csv(output_file, index=False)
    print(f"Updated dataset with forecasted values saved to {output_file}")
except Exception as e:
    print(f"Error while saving the file: {e}")


Enter the path to the .csv file: /content/ses - Sheet1.csv
Enter the column name for time series data: Sales
Enter the number of periods to forecast: 2
Enter alpha (0 < alpha <= 1): 0.5

Smoothed Results:
    Original         SES
0        120  120.000000
1        125  122.500000
2        130  126.250000
3        128  127.125000
4        135  131.062500
5        140  135.531250
6        138  136.765625
7        142  139.382812
8        145  142.191406
9        150  146.095703
10  Forecast  146.095703
11  Forecast  146.095703
Enter the output file name (with .csv extension): abc
Updated dataset with forecasted values saved to abc
